# Calculate the detection metrics of ORCNN

In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from numpy.linalg import inv
import os
import cv2
import random
import tifffile
from pyexcel_ods import get_data
import csv
import json
from tqdm import tqdm
import time
import seaborn as sns
import pandas as pd
import circle_fit as cf

# import some miscellaneous libraries
import visualize
import statistics

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog,MetadataCatalog
from detectron2.engine import DefaultTrainer
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 10,10

def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.show()

## Register the visible dataset

In [2]:
path = os.getcwd()
maindir = os.path.dirname(path)

In [3]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("broccoli_amodal_test", {}, maindir + "/datasets/train_val_test_files/orcnn/test/annotations.json", maindir + "/datasets/train_val_test_files/orcnn/test")

In [4]:
broccoli_amodal_test_metadata = MetadataCatalog.get("broccoli_amodal_test")
print(broccoli_amodal_test_metadata)

Metadata(evaluator_type='coco', image_root='/home/pieterdeeplearn/harvestcnn/datasets/train_val_test_files/orcnn/test', json_file='/home/pieterdeeplearn/harvestcnn/datasets/train_val_test_files/orcnn/test/annotations.json', name='broccoli_amodal_test')


In [5]:
dataset_dicts_test = DatasetCatalog.get("broccoli_amodal_test")

[12/08 16:58:13 d2.data.datasets.coco]: Loaded 487 images in COCO format from /home/pieterdeeplearn/harvestcnn/datasets/train_val_test_files/orcnn/test/annotations.json


## Initialize the image inference

In [6]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_orcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (broccoli)

cfg.OUTPUT_DIR = "weights/20201109_broccoli_amodal_visible"
cfg.MODEL.WEIGHTS = os.path.join(maindir, cfg.OUTPUT_DIR, "model_0007999.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.01
cfg.DATASETS.TEST = ("broccoli_amodal_test",)

predictor = DefaultPredictor(cfg)

## Calculate the detection metrics

In [7]:
# User input needed (directory, file locations, etc):
analysis_name = "orcnn_detection_metrics"
gtfile = os.path.join(maindir, "datasets/train_val_test_files/groundtruth_measurements_broccoli.ods") ## comment out if there is no ground truth file (also restart the kernel)
IoU_thres = 0.5
visualize_output = False

In [8]:
writedir = os.path.join(maindir, "results")

if not os.path.exists(writedir):
    os.makedirs(writedir)

try:
    gt = get_data(gtfile)
    gt_file_present = True
except:
    gt_file_present = False

csv_name = analysis_name + '.csv'
with open(os.path.join(writedir, csv_name), 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['image_id', 'number of gt masks', 'number of detected masks', 'number of true positives', 'number of false positives', 'number of false negatives', 'true positive on measured broccoli', 'false positive on measured broccoli', 'false negative on measured broccoli'])

ids_visualize = np.random.choice(np.arange(len(dataset_dicts_test)), 10)
random_images = []
random_imagenames = []

for i in tqdm(range(len(dataset_dicts_test))):
    
    # Load the RGB image
    imgname = dataset_dicts_test[i]["file_name"]
    basename = os.path.basename(imgname)
    img = cv2.imread(imgname)


    # Do the image inference and extract the outputs from Mask R-CNN
    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    classes = instances.pred_classes.numpy()
    scores = instances.scores.numpy()
    boxes = instances.pred_boxes.tensor.numpy()
    try:
        detection_num = len(boxes)
    except:
        detection_num = 0


    # Procedure to check whether we are dealing with ORCNN or MRCNN
    if "pred_visible_masks" in instances._fields:
        amodal_masks = instances.pred_masks.numpy()
        visible_masks = instances.pred_visible_masks.numpy()
    else:
        visible_masks = instances.pred_masks.numpy()


    # Visualize the masks
    if visualize_output:
        visualizer = Visualizer(img[:, :, ::-1], metadata=broccoli_visible_test_metadata, scale=0.8)
        vis = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
        imshow(vis.get_image()[:, :, ::-1])


    # Procedure to load the annotations (to calculate the amodal and the visible IoU)
    d = dataset_dicts_test[i]
    classes_annot = []
    amodal_masks_poly = []
    visible_masks_poly = []
    for k in range(len(d["annotations"])):
        classes_annot.append(d["annotations"][k]['category_id'])
        amodal_masks_poly.append(d["annotations"][k]['segmentation'])
        visible_masks_poly.append(d["annotations"][k]['visible_mask'])


    # Calculate the visible IoU    
    visible_masks_annot = visualize.make_mask_img(visible_masks_poly, d['height'], d['width'], "polylines")
    gt_num = visible_masks_annot.shape[0]
    iou_visible = statistics.calculate_iou(visible_masks_annot, visible_masks, np.array(classes_annot), classes)
    max_iou_visible = np.amax(iou_visible, axis=0)


    # Calculate the detection metrics
    TPs = np.where(max_iou_visible >= IoU_thres)[0].shape[0]
    FPs = detection_num - TPs
    FNs = gt_num - TPs
    
    
    # Calculate the detection metrics only on the measured broccoli heads
    gt_data_present = False

    if gt_file_present:
        for k in range(1, len(gt['groundtruth_measurements_broccoli'])):
            gt_data = gt['groundtruth_measurements_broccoli'][k]
            if gt_data:
                if gt_data[0] == basename:
                    gt_data_present = True
                    plant_id = gt_data[1]
                    x_center_gt = gt_data[3]
                    y_center_gt = gt_data[4]
                    coordinates_broccoli_gt = (x_center_gt, y_center_gt)


    # Find the broccoli head that belongs to the ground truth data
    distances = []
    if np.logical_and(boxes.size > 0, gt_data_present):
        for h in range(len(boxes)):
            box = boxes[h]
            x_center = box[0] + ((box[2] - box[0]) / 2)
            y_center = box[1] + ((box[3] - box[1]) / 2)
            distances.append(np.linalg.norm(np.asarray(coordinates_broccoli_gt) - np.asarray((x_center, y_center))))

        idx = np.asarray(distances).argmin()
        visible_IoU = max_iou_visible[idx]
    else:
        idx = []
        visible_IoU = np.nan

        
    # Calculate the detection metrics
    TP_broc = np.where(visible_IoU >= IoU_thres)[0].shape[0]
    FP_broc = 1 - TP_broc
    FN_broc = 1 - TP_broc
    
    
    # Store the results
    with open(os.path.join(writedir, csv_name), 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow([basename, gt_num, detection_num, TPs, FPs, FNs, TP_broc, FP_broc, FN_broc])

100%|██████████| 487/487 [01:14<00:00,  6.54it/s]
